In [9]:
%load_ext autoreload

%autoreload 2

from helper import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Create test set annoy file to use for evaluation metrics


In [1]:
# Function to return mean reciprocal rank from the model for the given list of question ids and associated list of duplicate "gold standard" question ids
# The score for an item in the question_ids list will be 1 if the first result given by the model matches the corresponding duplicate_ids item
# Searches k top results for each question
# Uses the basic BERT embedding from the helper.py file
def get_mrr_and_accuracy(question_ids, duplicate_ids, k, annoy_test_set_filename):
    mrr = 0.0
    accuracy = 0.0
    num_samples = len(questions)

    for i in num_samples:
        id = question_ids[i]
        duplicate_id = duplicate_ids[i]
        
        question = get_question_body_from_id(id)
        question_embedding = get_paragraph_embedding_bert(question)

        u = AnnoyIndex(768, 'angular')
        u.load(annoy_test_set_filename)
        nearest_i = u.get_nns_by_vector(question_embedding, k) # will return the index of the k nearest neighbor

        answer_rank = 0
        try:
            answer_rank = nearest_i.index(duplicate_id)
        except ValueError:
            # skip rest of iteration and don't update scores if the answer isn't found in the results
            continue
        
        mrr += 1.0 / answer_rank
        accuracy += 1 if answer_rank == 1.0 else 0
        
    mrr /= num_samples
    accuracy /= num_samples
    
    return mrr, accuracy
        

2023-03-31 21:42:47.485633: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 21:42:47.506119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 21:42:47.907632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-03-31 21:43:10.390596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [27]:
#feature choice will either be "title" or "body", but we can add a "both" option later
def get_all_question_embeddings1(feature_choice="body"):
    #embeddings will be array of shape [num_questions, 2], where each row will be: [question_id, question_embedding]
    embeddings = np.array([len(questions), 2])

    print("STARTING")

    index = 0
    for id, value in questions.items():
        print(index)
        question_vec = get_paragraph_embedding_bert(value[feature_choice])
        #embeddings[index] = float(id)
        embeddings[index, :] = [[id], [question_vec]]
        index += 1
    return embeddings

In [28]:
embeddings = get_all_question_embeddings1("body")

STARTING
0


/tmp/ipykernel_31323/4171296797.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings[index] = np.array(([id], [question_vec]))


ValueError: setting an array element with a sequence.

In [ ]:
print(len(embeddings))

In [ ]:
# set a seed 
# t.set_seed(487)
# we need this to tell what the original sentences are 

# t.build(10) # build 10 trees, allegedly the more the better  
# t.save('bert_embedding_unnormalize.ann')

# Find k-nearest neighbor
k = 3 # first set k = 5
test_sentence = "Why did Old English lose both thorn and eth?" # one of the questions on linguistic stack exchange
test_sentence_vector = encode_sentence(test_sentence)

f = 768
u = AnnoyIndex(f, 'angular')
u.load('bert_embedding1.ann') 
nearest_i = u.get_nns_by_vector(test_sentence_vector, k) # will return the index of the k nearest neighbor
nearest_word = [(i, questions[i]) for i in nearest_i] # 
nearest_word